In [1]:
import pandas as pd
import os
from tqdm import tqdm_notebook, tnrange
import numpy as np
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt

import scipy
from scipy.optimize import minimize
from scipy.sparse import csr_matrix, vstack, hstack

from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

import cvxpy as cp


module_path = os.path.abspath(os.path.join('..'))

%matplotlib inline

In [2]:
protein_df_PD1_test = pd.read_csv(module_path+'/data/harel/protein_df_PD1_test_feats.csv', index_col=0)

In [3]:
len(protein_df_PD1_test.T.columns)

15

In [4]:
proteins_df = pd.read_csv(module_path+'/data/harel/protein_df_PD1.csv', index_col=0)
protein_list = list(set(list(proteins_df["Gene_Names"])))
interactome_protein_set = set(np.loadtxt(module_path+
                            '/data/interactome/proteins_in_interactome.txt',
                            dtype=str))

In [5]:
intersection_proteins_to_keep = set()
semicolon_matches = ()
num_nan = 0
for protein in tqdm_notebook(protein_list):
    if protein in interactome_protein_set:
        intersection_proteins_to_keep.add(protein)
    elif len(protein.split(";")) > 1:
        
        if np.sum([x in interactome_protein_set for x in protein.split(";")]) > 1:
            print(protein.split(";"))

In [6]:
g = nx.read_edgelist(module_path+"/data/interactome/inbiomap_reduced.txt", 
                     data=(('confidence',float),))

In [7]:
intersection_proteins = set(g.nodes())
proteins_not_found = set(protein_list) - intersection_proteins
num_intersection_proteins = len(intersection_proteins)
num_orphan_proteins = len(proteins_not_found)

In [8]:
proteins_df["In_Interactome"] = list(proteins_df["Gene_Names"].isin(list(intersection_proteins)))
protein_new = proteins_df.sort_values(by="In_Interactome", ascending=False).reset_index(drop=True)

In [9]:
protein_number_to_names = list(protein_new["Gene_Names"])
protein_names_to_number = {}

for ind, protein in enumerate(protein_number_to_names):
    protein_names_to_number[protein] = ind

In [10]:
train_cols = [x for x in protein_new.columns if x not in protein_df_PD1_test.T.columns]

In [11]:
protein_test = protein_new[list(protein_df_PD1_test.T.columns) + ["Gene_Names", "In_Interactome"]]
protein_train = protein_new[train_cols]

A = protein_train.to_numpy()[:,:-2].T
Atest = protein_test.to_numpy()[:,:-2].T
Afull = protein_new.to_numpy()[:,:-2].T

In [12]:
A.shape, Atest.shape, Afull.shape

((59, 4249), (15, 4249), (74, 4249))

In [13]:
response_df = pd.read_csv(module_path+'/data/harel/response_df.csv', index_col=0)

In [14]:
patient_order_train = protein_train.columns[:-2]
y_raw_train = []

for patient in tqdm_notebook(patient_order_train):
    try:
        pfs = float(response_df[response_df["Sample ID"]==patient]["PFS time (months)"])
    except:
        new_patient = " ".join(patient.split("_"))
        try:
            pfs = float(response_df[response_df["Sample ID"]==new_patient]["PFS time (months)"])
        except:
            print(patient, new_patient)
            raise
        
    y_raw_train.append(pfs)
yraw_train = np.array(y_raw_train).reshape(-1,1)
scaler = StandardScaler()
scaler.fit(yraw_train)
y = scaler.transform(yraw_train.reshape(-1, 1)).reshape(-1)

In [15]:
patient_order_test = protein_test.columns[:-2]
y_raw_test = []

for patient in tqdm_notebook(patient_order_test):
    try:
        pfs = float(response_df[response_df["Sample ID"]==patient]["PFS time (months)"])
    except:
        new_patient = " ".join(patient.split("_"))
        try:
            pfs = float(response_df[response_df["Sample ID"]==new_patient]["PFS time (months)"])
        except:
            print(patient, new_patient)
            raise
        
    y_raw_test.append(pfs)
yraw_test = np.array(y_raw_test).reshape(-1,1)
scaler = StandardScaler()
scaler.fit(yraw_test)
y_test = scaler.transform(yraw_test.reshape(-1, 1)).reshape(-1)

In [16]:
patient_order = protein_new.columns[:-2]
y_raw_full = []

for patient in tqdm_notebook(patient_order):
    try:
        pfs = float(response_df[response_df["Sample ID"]==patient]["PFS time (months)"])
    except:
        new_patient = " ".join(patient.split("_"))
        try:
            pfs = float(response_df[response_df["Sample ID"]==new_patient]["PFS time (months)"])
        except:
            print(patient, new_patient)
            raise
        
    y_raw_full.append(pfs)
yraw_full = np.array(y_raw_full).reshape(-1,1)
scaler = StandardScaler()
scaler.fit(yraw_full)
y_full = scaler.transform(yraw_full.reshape(-1, 1)).reshape(-1)

In [17]:
nodlist = protein_train["Gene_Names"][:num_intersection_proteins]

L = nx.normalized_laplacian_matrix(g,nodelist=nodlist, weight="confidence")
U = nx.incidence_matrix(g,nodelist=nodlist, oriented=True, weight="confidence")

print(L.shape, U.shape)

Reg0 = np.eye(num_orphan_proteins+num_intersection_proteins)
Reg1 = csr_matrix(scipy.linalg.block_diag(
    L.todense(),np.eye(num_orphan_proteins)))
temp1 = hstack([U.T, csr_matrix(np.zeros((U.shape[1],num_orphan_proteins)))])
Reg2 = vstack ([temp1, csr_matrix(np.eye(num_intersection_proteins+num_orphan_proteins))])
Reg3 = csr_matrix(scipy.linalg.block_diag(
    L.todense(),np.zeros((num_orphan_proteins,num_orphan_proteins))))

Reg5 = csr_matrix(scipy.linalg.block_diag(np.zeros(num_intersection_proteins),
                                          np.eye(num_orphan_proteins)))

(3995, 3995) (3995, 165392)


In [18]:
def plot_regularization_path(lambd_values, beta_values):
    num_coeffs = len(beta_values[0])
    for i in range(num_coeffs):
        plt.plot(lambd_values, [wi[i] for wi in beta_values])
    plt.xlabel(r"$\alpha$", fontsize=16)
    plt.xscale("log")
    plt.title("Regularization Path")
    plt.grid()
    plt.show()

# Group lasso 

In [ ]:
x = cp.Variable(num_intersection_proteins+num_orphan_proteins)
alph = cp.Parameter(nonneg=True)

m = g.number_of_edges()
n = num_intersection_proteins+num_orphan_proteins
Areg = np.zeros((2,n))

alph.value = 0.1
reg = cp.norm2(Reg5*x)
for index, ed in tqdm_notebook(enumerate(g.edges())):
    end0 = protein_names_to_number[ed[0]]
    end1 = protein_names_to_number[ed[1]]
    Areg[0, end0] = 1
    Areg[1, end1] = 1
    reg += cp.norm2(Areg*x)
#     y = [x[0], x[10]]
#     reg += cp.norm2(x[end0]**2 + x[end1]**2)

cost = (1. / (2*A.shape[0])) * cp.sum_squares(A*x - y)+ alph* reg 

objective = cp.Minimize(cost)
prob = cp.Problem(objective)
prob.solve()

In [ ]:
print("a")